In [2]:
import sklearn, os, json, pickle, time, jieba, gensim, random
from scipy.spatial.distance import cosine
from collections import Counter
import pandas as pd
import numpy as np
from Bio.Cluster import kcluster
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline
import requests

In [3]:
content = pd.read_csv('qa_corpus.csv')
content = content.dropna(axis=0)
content = content.to_dict()
_q, _a = list(content['q'].values()), list(content['a'].values())
content = list(zip(content['q'].values(), list(content['a'].values())))

In [4]:
stop_words = set([el.strip() for el in open('百度停用词表.txt', 'r', encoding='utf8').readlines()])

In [5]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('cn.cbow.bin', binary=True, unicode_errors='ignore')

In [6]:
question = [[word2vec.wv[e] for e in jieba.lcut(el) if e not in stop_words and e in word2vec.wv.vocab] for el in _q]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\NACHOL~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.595 seconds.
Prefix dict has been built successfully.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [7]:
question_word = [[e for e in jieba.lcut(el) if e not in stop_words] for el in _q]

In [8]:
def cut_again(w, return_vec=True):
    if w in word2vec.wv.vocab:
        if return_vec:
            return [word2vec.wv[w]]
        return [w]
    else:
        for i in range(len(w) - 1):
            i += 1
            if w[:i] in word2vec.wv.vocab and w[i:] in word2vec.wv.vocab:
                if return_vec:
                    return [word2vec.wv[w[:i]], word2vec.wv[w[i:]]]
                return [w[:i], w[i:]]
        if return_vec:
            return None
        return None
cut_again('网银盾', 0)       

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


['网', '银盾']

In [9]:
question = []
for sentence in question_word:
    s = []
    for w in sentence:
        if w in word2vec.wv.vocab:
            s.append(w)
        else:
            vec = cut_again(w, False)
            if vec:
                s.extend(vec)
    question.append(s)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [10]:
question[:10]

[['补发', '网', '银盾'],
 ['代发', '工资'],
 ['帐单', '查询', '打印'],
 ['理财产品', '取', '号'],
 ['密码', '修改'],
 ['查询', '名下', '账户'],
 ['交易', '流水', '明细', '证明'],
 ['结售汇'],
 ['现金', '取款'],
 ['账户', '销户']]

In [11]:
def get_sentenses_vector(sentences, word2vec=word2vec, alpha=1e-3, u=None):
    vlookup = word2vec.wv.vocab  # Gives us access to word index and count
    Z = 0
    for k in vlookup:
        Z += vlookup[k].count # Compute the normalization constant Z
        
    def sif_embeddings(sentences, word2vec, alpha=alpha, Z=Z):

        vlookup = word2vec.wv.vocab  # Gives us access to word index and count
        vectors = word2vec.wv        # Gives us access to word vectors
        size = word2vec.vector_size  # Embedding size

        output = []

        # Iterate all sentences
        for s in sentences:
            count = 0
            v = np.zeros(size, dtype=np.float32) # Summary vector
            # Iterare all words
            for w in s:
                # A word must be present in the vocabulary
                if w in vlookup:
                    v += ( alpha / (alpha + (vlookup[w].count / Z))) * vectors[w]
                    count += 1 
            if count > 0:
                v /= count
            output.append(v)
        return np.vstack(output).astype(np.float32 )
    vector = sif_embeddings(sentences, word2vec)
    if u is None:
        pca = PCA(1)
        pca.fit(vector)
        u = pca.components_[0]
    vector -= np.multiply(np.multiply(u, u.T), vector)
    return vector, u

In [12]:
question_vec, u = get_sentenses_vector(question)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [13]:
category = 4
clusterid, error, nfound = kcluster(question_vec, category, dist='u',npass=100)
print(error, nfound)

17193.864719937636 1


In [14]:
dic = {i: [] for i in range(category)}
for i, c in enumerate(clusterid):
    dic[c].append(i)

In [15]:
print('sif + 余弦距离聚类效果:\n')
for k, v in dic.items():
    if v:
        print('category:', k)
        index_list = random.sample(v, 30)
        for i in index_list:
            print(_q[i])
        print('#' * 100)

sif + 余弦距离聚类效果:

category: 0
什么时候出生是摩羯座
因为我是人工智能的产物
中华民族的民族精神
打酱油好玩吗
怎么过圣诞
你有父母么
你知道这附近有哪些好玩的吗
暗恋是什么感觉
那我应该怎么办呢
儿童
六道骸?
败絮其中
晚间新闻
你是人工智能么
我怀疑你是男的哦
你比猪八戒还要蠢
你怎么这么笨
过端午节可以干什么
我打李新明
你不听话
那就这样吧
好孩子
你喜欢钓鱼吗
那场比赛是我输了
两弹一星分别是什么
闯红灯违法吗
你在学校开心吗
不要打我
你姐姐是谁
那我们谈恋爱吧
####################################################################################################
category: 1
你咋不呼叫了呢
你会不舒服吗
都谁教你这么贫的
些
你再学我就打你了
帮我弄一个qq好吗
总觉得现在的工作不适合我
书法有哪些种类
希望改变自己的什么？
你为什么喜欢过年
过一场生日的价格是多少
真棒讲的真好
今天干了点啥
这是你的吗
这些都不关你的事
看看你行吗
这么辛苦啊你
字怎么写得这么难看
朋友你真是有才
不要谢
好迪真好
被人相信的感觉真的好好啊。
你靓不靓
哇一米多高啊
您喜欢什么娱乐活动呢？
你认为在恋爱中谁会占主导地位，为什么？
可不可以快一点
你可会加减乘除
换一个笑话。
将来会怎样
####################################################################################################
category: 2
年金保险产品介绍
小麦。
个人网银购买理财产品
手机银行如何查询持有基金
跨行转账如何选择收款账户所属银行及开户行
一般转账的收款账户限制
flashget
今天能洗车吗？
离线缴税
我怎样查询信用卡积分？ 
什么是现汇账户
短信通知余额提醒
如何查询综合积分礼品兑换目录
广场
活期存款
能不能刷卡
两年的港币整存整取利率是多少
普通版网上银行修改网上支付功能的指引
退税一般需要多久才能完成
网银实物黄金（交易金）提货申请如何撤消
如何定制电话银行快捷菜单
社会医保中哪些费用可以在建信人寿申请赔付
手机银行能否购买保险
补发网

In [16]:
def get_the_cluster_center(sentences):
    v= np.zeros(sentences[0].shape)
    for el in sentences:
        v += el
    v /= len(sentences)
    return v

In [17]:
center = {}
for k, v in dic.items():
    if v:
        center[k] = get_the_cluster_center([question_vec[el] for el in v])

检查样本和我计算的中心点的cosine距离

In [18]:
for k, v in dic.items():
    print('category:', k)
    index_list = random.sample(v, 10)
    vec_list = [(_q[el], question_vec[el]) for el in index_list]
    for sentence, el in vec_list:
        cosine_distance = [cosine(el, center[0]), cosine(el, center[1]), cosine(el, center[2]), cosine(el, center[3])]
        print(cosine_distance.index(min(cosine_distance)), sentence)
    print('#' * 100)

category: 0
0 元宵节祝福
0 个性
0 你傻冒
0 你能够抵挡黑客攻击么
0 中国在机器人足球国际赛场上金牌零的突破是在哪一年
0 中华人民共和国国旗五星红旗的设计者是谁
0 你男朋友做什么的
0 您捶啊
0 浮游城
0 冬至一定要吃什么
####################################################################################################
category: 1
1 苦逼的人生呀
1 今天很烦
1 你有什么优点？
1 时间是最好的橡皮擦！
1 被人爱
1 你一天有哪些工作
1 数学好吗
1 这边有一个
1 我美吗？
1 我要去洗下头发
####################################################################################################
category: 2
2 如何查询（修改）信用记录（报告）
2 商城中分期的商品是否有手续费和利息？分期还款是什么时候？
2 善融商务中购买商品是否收取送货费
2 数字医院主要服务什么？
2 结算通卡能否开通手机银行
2 电子商业汇票追索介绍
2 信用卡账单分期能否累计积分
2 如何使用手机银行“日历”功能
2 我家在XXXXX
2 借记卡申领
####################################################################################################
category: 3
3 最长多长时间
3 您想问啥
3 好啊，有什么好吃的招待
3 他咋样
3 请注意你的仪态
3 貌似不错
3 好了不能跟你這種白癡聊我走了?
0 后来呢
0 BOC
3 楼下有咖啡厅吗
####################################################################################################


C:\Anaconda3\lib\site-packages\scipy\spatial\distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [19]:
content

[('补发网银盾', '您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。'),
 ('代发工资', '您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。'),
 ('对帐单查询打印', '您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。'),
 ('理财产品取号', '请说出您需要办理的业务'),
 ('密码修改', '请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。'),
 ('查询名下所有账户', '您可以到智慧柜员机办理此项业务，如需到柜台办理，请刷身份证取号'),
 ('交易流水明细证明', '您可以到智慧柜员机办理此项业务，如需到柜台办理，请刷身份证取号'),
 ('个人结售汇',
  '电视墙上有实时汇率表哦，您可以看看，如果您需要购汇或换汇，请到自助柜员机办理，如果您要取现钞，请刷身份证取号到柜台办理。如您使用护照或其他证件，可以请大堂经理过来帮您哦'),
 ('现金取款', '人民币百元钞2万以内可直接在ATM机办理，2万以上请刷身份证取号到柜台办理哦。'),
 ('账户销户', '您要办理的是个人转账、挂失、销户、修改密码业务。请刷身份证，小龙人来帮您取个号吧。'),
 ('转账业务',
  '5万元以内转账，可以直接在ATM办理；五万以上请刷身份证取号至柜台办理。您也可以通过网上银行或手机银行办理。我行手机银行所有转账手续费全免！'),
 ('现金存款', '人民币百元钞2万以内可直接在自助柜员机办理，2万以上请刷身份证取号去柜台办理。'),
 ('贵金属业务', '购买纸黄金可以在手机银行客户端签约购买，方便快捷；购买实物黄金请刷身份证取号到柜台办理，详情可以咨询我行客户经理。'),
 ('个人账户管理', '您要办理的是个人开户、签约及理财业务，小龙人来帮您取个号吧。'),
 ('开特色卡', '您要办理的是个人开户、签约及理财业务，小龙人来帮您取个号吧。'),
 ('吞卡证明', '您要办理的是个人开户、签约及理财业务，小龙人来帮您取个号吧。'),
 ('补打发票', '您要办理的是个人开户、签约及理财业务，小龙人来帮您取个号吧。'),
 ('查询并取款', '请您刷身份证取号！'),
 ('本卡转本卡', '请您刷身份证取号！'),

In [20]:
content_dict = {i: [q, sentence_vec, a] for i, ((q, a), sentence_vec) in enumerate(zip(content, question_vec))}

In [21]:
content_dict[32510]

['园区介绍',
 array([ 4.45342697e-02,  2.71084700e-02, -3.28355171e-02,  8.13645776e-03,
         1.49567589e-01, -7.51185371e-03, -8.28081742e-02, -1.10379979e-01,
         1.53177902e-01,  1.18086264e-01,  1.42130312e-02,  3.41181904e-02,
         1.66453108e-01,  2.46002693e-02, -2.94489376e-02,  7.16126412e-02,
        -9.53726619e-02, -7.68371075e-02, -1.39072165e-01,  8.36441442e-02,
         8.71790573e-03, -1.02977589e-01, -3.20080034e-02, -8.69897902e-02,
        -7.57195204e-02,  2.97566373e-02, -1.38453886e-01,  9.67052132e-02,
        -9.08291936e-02, -3.97544261e-03, -7.77133834e-03,  1.03140466e-01,
        -9.52428281e-02,  3.99576558e-04,  9.65967700e-02,  2.49986768e-01,
         4.76463139e-02, -1.41869828e-01,  7.35575110e-02, -1.93315335e-02,
        -1.07074142e-01,  1.16744041e-01,  9.64590907e-03, -1.07559957e-01,
        -8.37540627e-02, -1.30868062e-01,  1.41344406e-02, -5.96195012e-02,
         3.10243331e-02, -9.48195234e-02, -8.24166536e-02, -1.13492996e-01,
   

In [22]:
word_list = sum(question_word, [])
word_index = {word: [] for word in word_list}

In [23]:
for i, word_list in enumerate(question_word):
    for word in word_list:
        if word in word_index:
            word_index[word].append(i)

In [24]:
word_index['补发']

[0, 161, 720, 31956, 32396]

In [45]:
sentence_index = {}
threshold = 1024
question_id_vec = {i: el for i, el in enumerate(question_vec)}

def get_sentence_index(question_id_vec, key_list):
    
    question_id_vec_keys = list(question_id_vec.keys())  # [0, 9, 11, 45]
    clusterid, error, nfound = kcluster(list(question_id_vec.values()), 4, dist='u',npass=100)
    counts = Counter(clusterid)
    for category, count in counts.items():
        _question_id_vec_keys = set([key for key, _clusterid in zip(question_id_vec_keys, clusterid) if _clusterid == category])
#         category_index_set = set([i for i, el in enumerate(clusterid) if el == category])  # {0, 1, 4, 5, 6, 8, 9}
#         _question_id_vec_keys = set([el for i, el in enumerate(question_id_vec_keys) if i in category_index_set])
#         assert _question_id_vec_keys == __question_id_vec_keys, 'asdfasdfasdfasdfasdf'
        _question_id_vec = {k: v for k, v in question_id_vec.items() if k in _question_id_vec_keys}

        vec_list = list(_question_id_vec.values())
        center_vec = tuple(get_the_cluster_center(vec_list))
        
        _key_question_id_vec = sentence_index
        for key in key_list:
            _key_question_id_vec = _key_question_id_vec[key]
        
        if count > threshold:
            print(count)
            _key_question_id_vec[center_vec] = {}
            get_sentence_index(_question_id_vec, key_list + [center_vec])
        else:
            _key_question_id_vec[center_vec] = list(_question_id_vec_keys)

In [46]:
get_sentence_index(question_id_vec, [])

11289
2979
2978
3031
2301
9326
2188
2465
2789
1325
1884
5143
2019
1347
6767
2310
2184
1421


In [71]:
def prin(sentence_index, i):
    for k, v in sentence_index.items():
        if type(v) == dict:
            prin(v, i + 1)
        else:
            print(i, ':', len(v))
prin(sentence_index, 0)

2 : 903
2 : 540
2 : 783
2 : 753
2 : 668
2 : 628
2 : 829
2 : 853
2 : 847
2 : 863
2 : 609
2 : 712
2 : 707
2 : 463
2 : 845
2 : 286
2 : 569
2 : 874
2 : 488
2 : 257
2 : 744
2 : 764
2 : 558
2 : 399
3 : 352
3 : 223
3 : 654
3 : 96
2 : 464
2 : 500
2 : 500
2 : 582
2 : 350
2 : 325
2 : 627
2 : 613
2 : 540
2 : 555
2 : 311
2 : 332
2 : 460
2 : 297
2 : 258
1 : 950
1 : 827
2 : 637
2 : 642
2 : 599
2 : 432
1 : 852
2 : 378
2 : 561
2 : 738
2 : 507
2 : 335
2 : 379
2 : 597
2 : 110


In [197]:
def get_sentence_vec_u(sentence, u):
    def cut_again(w, return_vec=False):
        if w in word2vec.wv.vocab:
            if return_vec:
                return [word2vec.wv[w]]
            return [w]
        else:
            for i in range(len(w) - 1):
                i += 1
                if w[:i] in word2vec.wv.vocab and w[i:] in word2vec.wv.vocab:
                    if return_vec:
                        return [word2vec.wv[w[:i]], word2vec.wv[w[i:]]]
                    return [w[:i], w[i:]]
            if return_vec:
                return None
            return None
        
    def get_sentence_vec(sentence_word_list, u, alpha=1e-3):
        vlookup = word2vec.wv.vocab  # Gives us access to word index and count
        vectors = word2vec.wv        # Gives us access to word vectors
        size = word2vec.vector_size  # Embedding size
        Z = 0
        for k in vlookup:
            Z += vlookup[k].count # Compute the normalization constant Z
        count = 0
        v = np.zeros(size, dtype=np.float32) # Summary vector
        for w in sentence_word_list:
            if w in vlookup:
                v += (alpha / (alpha + (vlookup[w].count / Z))) * vectors[w]
                count += 1 
        if count > 0:
            v /= count
        v -= np.multiply(np.multiply(u, u.T), v)
        return v
        
    s = []
    for w in [e for e in jieba.lcut(sentence) if e not in stop_words]:
        if w in word2vec.wv.vocab:
            s.append(w)
        else:
            words = cut_again(w, False)
            if words is not None:
                s.extend(words)
    sentence_vec = get_sentence_vec(s, u)
    
    return sentence_vec

In [97]:
def get_10_vec(sentence, sentence_index):
    sentence_vec = get_sentence_vec_u(sentence, u)
    while 1:
        if type(sentence_index) == dict:
            distance = sorted([(cosine(k, sentence_vec), v) for k, v in sentence_index.items()], key=lambda x: x[0])
            sentence_index = distance[0][1]
        else:
            sentence_list = sentence_index  # [10, 24589, 24590, 26641, 20, 21, 22, 23, 24602, 24605, ...]
            break
    similarity_list = [(sentence_id, cosine(content_dict[sentence_id][1], sentence_vec)) for sentence_id in sentence_list]
    similarity_list = sorted(similarity_list, key=lambda x: x[1])
    similarity_list = similarity_list[:10]
    similarity_list = [content_dict[i[0]] for i in similarity_list]
    return similarity_list

In [213]:
def get_10_word(sentence, word_index, n=10):
    sentence = [el for el in jieba.lcut(sentence) if el not in stop_words]
    similarity_sentences = set(sum([word_index[word] for word in sentence if word in word_index], []))  # {16015, 16016, 20495,..}
    similarity_sentences = [content_dict[el] for el in similarity_sentences]
    random.shuffle(similarity_sentences)
    similarity_sentences = [(similarity_sentence, len(set(sentence) & set([el for el in jieba.lcut(similarity_sentence[0]) if el not in stop_words]))) for similarity_sentence in similarity_sentences]
    similarity_sentences = sorted(similarity_sentences, key=lambda x: x[1])[:n]
    similarity_sentences = [el[0] for el in similarity_sentences]
    return similarity_sentences

In [214]:
bussiness_sentence_vec = (0.007251556825472989, -0.03578881006509725, -0.004198648500922097, -0.014993346668505133, 0.006125576159038294, -0.047159025765702356, -0.00815700508406887, -0.059917636587682295, 0.09358155436239915, 0.016037753164907957, -0.04334697983523884, -0.0004120683979386997, 0.015234787523622345, 0.011328430526555112, -0.04679575556697047, 0.01970830682712139, -0.04226480140991886, -0.027740232060393975, -0.0028206794158207125, 0.0444531150912312, -0.05180890024589371, 0.024336792811110602, -0.045165106640602824, 0.031988889592018656, -0.04493665619664707, -0.010370208706053943, -0.04371441599918962, -0.03153986639824259, -0.02559948122880422, -0.03388404668167708, -0.02669680131213664, 0.07470535706142517, -0.06412229755075935, -0.025388148380635196, 0.030878696312355132, 0.11332641431435228, -0.014355638948641013, -0.017179489437837713, -0.024013266656595287, -0.12544092564351925, -0.0006127882507166399, -0.02043552514810774, 0.023812293188497556, -0.07068558646568512, -0.06107162254521964, -0.038834967476057344, -0.059257032801024456, 0.08379323652478186, -0.028999517853309318, -0.00400975671796284, 0.000759218510582938, 0.008601261702800856, 0.003316914715588399, -0.1242125417794263, -0.016288522226197512, 0.00019520291089652233, -0.01046884797249675, -0.0826888705550761, 0.031456465579784654, 0.06297601985020078, 0.01888696495734343, -0.009966640082171078, -0.042696265410029924, 0.03748410826543866, 0.010263746344731312, 0.025925084232333886, 0.03822944666247273, 0.009954209177661677, -0.03110043695964332, -0.04210792858405347, 0.01590459192052647, 0.03207632467520545, -0.0510828965446149, 0.032815898862885294, -0.0006862096014627771, -0.043633716516712745, 0.035895513866294804, 0.05565579030118109, 0.04206131654260145, 0.042221841833769286, -0.012655289143437061, -0.031277188345981366, 0.005263504203850987, 0.020106689608794345, 0.08319185325982982, -0.016092275741639437, -0.03862014576377242, 0.009818188314407858, 0.07966988736444419, -0.0203955972026009, -0.005504344404643587, 0.029332330735184382, -0.0502417095940903, -0.019251238223077515, 0.020113521771305227, -0.051347506140533225, -0.05163111580421589, -0.013614045846245426, 0.024625602008940706, 0.05434533013348808, -0.013551163052994602, -0.08890922134928245, -0.06048037319234706, 0.11319519071963798, 0.00016457188834874127, 0.08473153745764392, -0.03192928527157878, 0.08095525238095357, -0.0862768166554266, 0.009844078095358524, -0.006161218426427684, 0.048661789622079295, -0.0045876575741791775, 0.011933703656961394, -0.006540860967692174, 0.023360502480077303, 0.07792101580552682, 0.09336956715111273, -0.028969602961847172, 0.02670548268691429, -0.039187899685402705, -0.04153847678437111, -0.01056814872403892, 0.010580849154505327, -0.0010627771035765228, -0.046724657286785716, -0.009397915202396478, 0.05538341524430214, 0.03430896771839775, 0.0399343957884086, 0.03258947404407985, 0.08019752417288357, 0.042313266019355554, -0.027296764503745536, 0.08383263516675368, -0.10751418281773219, 0.022410963774804057, -0.08427585863462965, -0.027167343005832685, 0.0011303053012945677, -0.08067395552168473, 0.08194681502533585, -0.0744239352445942, -0.012221484378992075, -0.052547837943566614, 0.023150927998883946, 0.02065011795345933, -0.07853860132742155, -0.009696501040043937, -0.058498642778411915, 0.05628694478590305, -0.021310556594997934, -0.04758664173125768, 0.0015194939044870575, 0.05769949751223629, -0.0042902796592202445, 0.08624277514542143, -0.015411265528447722, 0.06565993873443081, 0.06719719863478955, 0.04793950310116987, 0.03621650025468067, -0.02505632101176515, -0.031031053181277926, -0.0533425903882154, -0.045159229514468255, 0.009209520108249044, 0.038176170696384286, 0.03137423883455809, 0.0014405418070535065, 0.04041419446890337, 0.04653616639026978, 0.041869716852386725, -0.008653517016339715, 0.06175922953874839, 0.032170848901264155, 0.03259298264463679, -0.00290215408908087, 0.01680549857906886, 0.07218155790234451, 0.08889628295279975, 0.032504775573804205, -0.04036915452676608, -0.061083675660208954, -0.028003408786397847, 0.058821458752284006, -0.05195994157856072, -0.004351994754004177, -0.009016894969830866, 0.026633790441909763, 0.11148543814197931, -0.03672305890384234, -0.006529903852860726, -0.11497252839438822, -0.007683700719945254, -0.031047320010288548, 0.020851816497316357, -0.013896013576493359, 0.004339421723289956, -0.03234003962364558, -0.021018518556911656, 0.04158976402965067, 0.0032580855181845833, -0.050312127182845695, 0.026246440766076365, 0.11392602558226288, 0.02238097776279663, 0.013387593093163997, -0.060402544717821194, -0.1058102658118682, -0.021726399167247005, 0.030215665846117415, 0.08679538087840474, -0.0047634276139174765, -0.049057265431107853, 0.06936396757636977, -0.12683696550528883, 0.010524735153967619, 0.06235872957087867, 0.041570992069102586, 0.05696232837332782, -0.013621192932796853, -0.021465157923502003, -0.08481537364353611, -0.0051253282865091725, 0.024577386655666986, -0.02237891104542795, 0.04861122140009159, -0.06520322860641324, 0.011348037402898059, -0.01224997237651745, -0.005415486214629347, 0.12900057696388825, -0.04424694380819421, -0.027861494873828645, -0.04055326020956216, 0.06393393741285396, 0.006188154299182819, 0.037552622889745, -0.004475973942108341, -0.013613584903881017, -0.0013852275723159968, -0.09266711674509158, -0.014104753147757288, 0.01988439756141075, -0.008796932525315668, -0.011617150296228432, 0.09665374773024735, 0.06326745909799344, -0.016128099782974827, 0.030088272828296014, -0.027434343432556824, -0.017503900575842522, 0.05433599321276299, -0.047659565547060564, 0.04906692709544276, 0.004335814869920424, -0.008414522333526075, 0.011189901103661038, 0.09261281058577354, -0.06601218030391207, -0.021758417069934165, -0.059065375207893345, 0.00896374241958348, -0.03175011906552384, 0.0015741971361628475, 0.06410941931939153, 0.04537227743809031, 0.009052404921376407, -0.002001733285647283, -0.07203862749169243, -0.05138281570185481, -0.03692554054984275, 0.0922562868228727, -0.009249855382067806, 0.015289690140784297, 0.060059352495133354, -0.014650897992901715, -0.042192589415743074, -0.10034377368538745, -0.058860274442684594, -0.010852867286821756, 0.09030525378584754, 0.021681377499326023, -0.031640024151322305, -0.021050647221264984, 0.022006217937185685, -0.046766191600236186, -0.1002737482529174, -0.024513455916478154, 0.014921890913599115, -0.020373601925891556, -0.0301275951113418, -0.07448219696604047, -0.008697869596388713, -0.09935055684488052, -0.032619861996771374, 0.030303708066910816, -0.02630171214865093, 0.009817507004420528)
vec_list = list(sentence_index.keys())
vec_list2 = [cosine(vec, bussiness_sentence_vec) for vec in vec_list]
bussiness_sentence_vec = vec_list[vec_list2.index(min(vec_list2))]
print(bussiness_sentence_vec)

(0.007251556825472989, -0.03578881006509725, -0.004198648500922097, -0.014993346668505133, 0.006125576159038294, -0.047159025765702356, -0.00815700508406887, -0.059917636587682295, 0.09358155436239915, 0.016037753164907957, -0.04334697983523884, -0.0004120683979386997, 0.015234787523622345, 0.011328430526555112, -0.04679575556697047, 0.01970830682712139, -0.04226480140991886, -0.027740232060393975, -0.0028206794158207125, 0.0444531150912312, -0.05180890024589371, 0.024336792811110602, -0.045165106640602824, 0.031988889592018656, -0.04493665619664707, -0.010370208706053943, -0.04371441599918962, -0.03153986639824259, -0.02559948122880422, -0.03388404668167708, -0.02669680131213664, 0.07470535706142517, -0.06412229755075935, -0.025388148380635196, 0.030878696312355132, 0.11332641431435228, -0.014355638948641013, -0.017179489437837713, -0.024013266656595287, -0.12544092564351925, -0.0006127882507166399, -0.02043552514810774, 0.023812293188497556, -0.07068558646568512, -0.06107162254521964

In [215]:
def get_1_from_20(vec_list, bool_list):
    vec_list_sentence, bool_list_sentence = [el[0] for el in vec_list], [el[0] for el in bool_list]
    common = set(vec_list_sentence) | set(bool_list_sentence)
    if common:
        for el in vec_list:
            if el[0] in common:
                return el[2]
    return vec_list[0][2]

In [216]:
def related_to_bussiness(sentence, sentence_index, bussiness_sentence_vec=bussiness_sentence_vec):
#     vec_list = list(sentence_index.keys())
#     vec_list2 = [cosine(vec, bussiness_sentence_vec) for vec in vec_list]
#     bussiness_sentence_vec = vec_list[vec_list2.index(min(vec_list2))]
    
    sentence_vec = get_sentence_vec_u(sentence, u)
    bussiness_cosine = cosine(sentence_vec, bussiness_sentence_vec)
    others_cosine = [cosine(sentence_vec, vec) for vec in vec_list if vec != bussiness_sentence_vec]
    if bussiness_cosine < min(others_cosine) and bussiness_cosine < 0.7:
        return True
    return False 

In [217]:
def get_random_answer(sentence):
    return f'{sentence}这个问题我还需要再学习一下'

检查结果

词向量的最接近的10个

In [151]:
t = time.time()
([print(i, el[0]) for i, el in enumerate(get_10_vec('转账手续费是多少', sentence_index))])
print(time.time() - t)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


0 转账手续费
1 手机银行转账手续费咨询
2 一般转账的金额限制
3 转账业务
4 我在网上办理跨行转账可以转账到省外吗？
5 什么是ATM转账还款？
6 定制转账中周期转账的转账条件
7 我发送短信向结算通卡账户转账，为什么收到的转账确认短信中的手续费金额旁注明了“收款方支付”？ 
8 银证转账办理时间
9 手机银行转账汇款的历史转账记录能否保留
0.5076799392700195


In [150]:
t = time.time()
([print(i, el[0]) for i, el in enumerate(get_10_vec('今天天气真几把热', sentence_index))])
print(time.time() - t)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


0 今天真热啊
1 今天真热
2 真的，我说的真的
3 不是真的
4 看来你你真的好蠢
5 真的啊我要哭了
6 你真2你真20
7 我是真的回家
8 我真的爱你
9 双子的异性缘真的是好到不行，自带气场的那种呀。
0.493011474609375


In [149]:
t = time.time()
([print(i, el[0]) for i, el in enumerate(get_10_vec('你是啥学历', sentence_index))])
print(time.time() - t)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


0 你什么学历
1 代理他人存款的客户身份识别要求
2 个人保障性住房贷款的业务简介
3 特殊客户存款人本人在国外的情况
4 金融IC卡的有效期
5 个人保障性住房贷款的业务特色
6 出境游保证金证明的被证明人要求
7 信托业务简介
8 个人出境游保证金证明的业务介绍
9 存省息业务的办理对象
0.46767640113830566


布尔判断的最接近的10个

In [148]:
t = time.time()
([print(i, el[0]) for i, el in enumerate(get_10_word('转账手续费是多少', word_index))])
print(time.time() - t)

0 定制转账中所说的周期转账是什么
1 定制转账业务规则
2 中信银行异地跨行取款手续费
3 转账提交后可以修改附言内容吗
4 B股集中式银证转账业务特色
5 银衍转账的介绍
6 您在网银渠道首次开通短信转账或缴费时，网银返显手机号可以修改吗？
7 人民币转账退款原因查询
8 如何取消短信转账功能
9 信用卡账单分期可以申请提前还款吗，分期手续费如何计算？
0.05421304702758789


In [153]:
t = time.time()
([print(i, el[0], el[2]) for i, el in enumerate(get_10_word('今天天气真几把热', word_index))])
print(time.time() - t)

0 那我真打你 哇~~~不要了~我很听话的
1 今天的饭真难吃 是吗？下次少吃点。
2 今天真郁闷: 怎么啦！
3 帮我热一下饭 怎么热？
4 真小气！ 已经不能用小气形容了
5 你真聪明 您这么说我会害羞的哦
6 你真可爱 谢谢，我都害羞了
7 今天真倒霉把钱包丢了 我也丢过
8 你真悲哀 做人要乐观开心，机器人也一样
9 我想说，生活真美好。 生活中的美好真是处处都能发现呢。
0.015707969665527344


In [152]:
t = time.time()
([print(i, el[0], el[2]) for i, el in enumerate(get_10_word('你是啥学历', word_index))])
print(time.time() - t)

0 什么学历 小学没毕业
1 你的学历 我是大一的，说起来也算个学霸呢～
2 你什么学历 文盲
0.001010894775390625


把词向量和布尔判断的20个放到一起, 选出回答的一个

In [212]:
get_1_from_20(get_10_vec('转账手续费是多少', sentence_index), get_10_word('转账手续费是多少', word_index, 1000))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


'我行手机银行转账手续费全免，其余见附表'

检查是否是业务相关问题的判断准确率的precision和recall

In [228]:
i = 0
is_business = []
not_business = []
sample = random.sample([el[0] for el in content_dict.values()], 500)
while len(is_business) < 100 or len(not_business) < 100:
    for el in sample:
        b = related_to_bussiness(el, sentence_index)
        if b:
            is_business.append(el)
        else:
            not_business.append(el)

print('related_to_bussiness')
print('\n'.join(random.sample(is_business, 100)))
print('#' * 100, '\nnot related_to_bussiness')
print('\n'.join(random.sample(not_business, 100)))

related_to_bussiness
交易明细疑义查询
通用盾（音频盾）的介绍
网上支付限额是否可以更改？
些商户支持积分商户兑换?
账号支付开通
借记卡换卡业务介绍
页面上方的所在城市可以切换到其他城市吗？
数字医院方案还有什么产品组合？
他人代销活期存款账户
“振兴演艺”可以为相关客户提供哪些服务？
你有电子邮件地址吗
什么是储蓄存款继承
银银合作第三方存管业务简介
提示“0250B2107003您输入的账(卡号)与您选择的交易类别不匹配，请确认您的输入是否有误，或拨打95533咨询”是什么意思
澳大利亚元活期存款利率是多少
信用卡结算币种前的星号什么意思
离岸美元汇款手续费优惠活动内容
银行微银行提供哪些服务
办理缴费
什么是电汇汇出业务
电话银行办理自助缴费
若我旧卡本期已扣过费，换新卡后还会重复扣短信服务费吗？
用户配置的功能介绍
电话银行预约转账是否有笔数限制
储蓄国债(电子式)的特点
住宅信息的修改渠道
我手机停机了
未登录过PC版，是否可以在手机客户端进行首次登录？ 
“南商龙卡”信用卡可享受哪些权益
电子渠道结售汇币种介绍
网上银行信用卡积分合并流程是什么？
点击“忘记网银用户名”时系统报错“输入的卡号尚未设置昵称”怎么办
如何撤销网上银行购汇还款？
开放式理财产品为什么没有赎回按钮
“上戏龙卡”附属卡的卡面与主卡是否一样
“深航龙卡”的发卡对象
自助票据机用户号登录的介绍
原手机无法使用，如何解绑手机银行客户端
境外电汇汇入的特点是什么
网银注销后，我是否可以原来网银盾再次开通网上银行？
私人网银教育规划功能的操作步骤
预约转账功能的开通渠道
电话银行公积金账户的查询方法
电话银行转账至他人账户
票据是否有支付密码
你有没有电子邮件
什么是ATM转账还款？
没有银行卡能否购买基金
可办理外币光票托收的票据种类
登录网银提示“系统忙，请稍后再试”怎么办
登录网银时显示“服务器没有接收到您的证书”怎么办
个人网银“银期管理”中的“变更账户”是什么
我们保持电话联系
为什么会收到信用卡升级的邀请短信
如何通过网银投保
“陆港通龙卡”的有效期
公司外汇买卖办理时间
什么是预授权申请
PAD安全性保障
我可以使用上行短信方式取消基金短信服务吗？
手机银行如何缴党费
金融IC卡降级交易的介绍
智慧柜员机上修改手机号
购汇还款是否需要手续费
财富之星介绍


In [218]:
pickle.dump(sentence_index, open('sentence_index', 'wb'))
pickle.dump(word_index, open('word_index', 'wb'))
pickle.dump(bussiness_sentence_vec, open('bussiness_sentence_vec', 'wb'))
pickle.dump(content_dict, open('content_dict', 'wb'))
pickle.dump(u, open('u', 'wb'))